In [1]:
# imports
import geopandas as gpd
import pandas as pd
from shapely import wkt
import random # used for colors in ant paths
import folium
from folium import plugins
from folium import FeatureGroup
from folium.plugins import FloatImage
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import Layout

In [2]:
# load requests
trips = pd.read_csv(r'.\requests.csv')
trips['path'] = trips['path'].apply(wkt.loads)
trips['origin'] = trips['origin'].apply(wkt.loads)
trips['destination'] = trips['destination'].apply(wkt.loads)
trips_gdf = gpd.GeoDataFrame(trips.copy(),crs='epsg:4326',geometry='path')

In [3]:
# load dispatches (pairs)
dispatches = pd.read_csv(r'.\dispatches_total.csv')
dispatches['origin1'] = dispatches['origin1'].apply(wkt.loads)
dispatches['destination1'] = dispatches['destination1'].apply(wkt.loads)
dispatches['origin2'] = dispatches['origin2'].apply(wkt.loads)
dispatches['destination2'] = dispatches['destination2'].apply(wkt.loads)
dispatches['path'] = dispatches['path'].apply(wkt.loads)

In [4]:
def plot_map2(ant, tot):
    m = folium.Map(location=[40.6401, 22.9444], tiles="Stamen Toner", zoom_start=11.5)

    trips_ = FeatureGroup(name='Trips')
    origins = FeatureGroup(name='Origins')
    destinations = FeatureGroup(name='Destinations')
    requests = FeatureGroup(name='Total requests')

    
    m.add_child(requests)
    m.add_child(origins)
    m.add_child(destinations)
    m.add_child(trips_)

    #Pairs
    for ind in ant.index:
        c = '#'+"%06x" % random.randint(0, 0xFFFFFF)
        points_xy = list(ant.loc[ind,'path'].coords)
        points_yx = []
        popup='Dispatch:{} Req.1:{} Req.2:{} {}'.format(ind,ant.loc[ind,'req1'],ant.loc[ind,'req2'],ant.loc[ind,'method'])
        for p in points_xy:
            points_yx.append((p[1],p[0]))
        folium.plugins.AntPath(points_yx, color=c, weight=3.5, opacity=1,popup=popup, delay='2000').add_to(trips_)

        folium.CircleMarker(
        location=[ant.loc[ind,'origin1'].y, ant.loc[ind,'origin1'].x],
        popup= 'Dis:{}\nReq.:{}\nO1'.format(ind, ant.loc[ind,'req1'][0:4]),
        color=c,fill=True,fill_opacity=1).add_to(origins)


        folium.CircleMarker(
        location=[ant.loc[ind,'destination1'].y, ant.loc[ind,'destination1'].x],
        popup= 'Dis:{}\nReq.:{}\nD1'.format(ind, ant.loc[ind,'req1'][0:4]),
        color=c,fill=True,fill_opacity=0.5).add_to(destinations)

        folium.CircleMarker(
        location=[ant.loc[ind,'origin2'].y, ant.loc[ind,'origin2'].x],
        popup= 'Dis:{}\nReq.:{}\nO2'.format(ind, ant.loc[ind,'req2'][0:4]),
        color=c,fill=True,fill_opacity=1).add_to(origins)

        folium.CircleMarker(
        location=[ant.loc[ind,'destination2'].y, ant.loc[ind,'destination2'].x],
        popup= 'Dis:{}\nReq.:{}\nD2'.format(ind, ant.loc[ind,'req2'][0:4]),
        color=c,fill=True,fill_opacity=0.5).add_to(destinations)

    # Total requests
    def highlight_function(feature):
        return {"fillColor": "#800000", "color": "#800000", "weight": 5}
    
    folium.GeoJson(
    tot[['id_request', 'path']].to_json(),
    name='Requests',
    show=True,
    style_function=lambda x: {
        'fillColor': '#0bba83',
        'color': '#0bba83',
        'weight': 5,
        'fillOpacity':0.005,
        'opacity':0.7
    },
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['id_request'],
        aliases=['id_request'],
    ),
    ).add_to(requests)

    for ind in tot.index:
        folium.CircleMarker(
        location=[tot.loc[ind,'origin'].y, tot.loc[ind,'origin'].x],
        color='#0bba83',radius=8, fill=True,fill_opacity=1).add_to(requests)

        folium.CircleMarker(
        location=[tot.loc[ind,'destination'].y, tot.loc[ind,'destination'].x], 
        color='#0bba83',radius=8, fill=True,fill_opacity=0.5).add_to(requests)

    m.add_child(folium.map.LayerControl(collapsed=True))
    folium.plugins.Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen').add_to(m)
    
    
    
    return m

In [7]:
# time windows
hours = []
for t in range (8,10):
    hours.append(str(t))
# print(hours)
for k in range (len(hours)):
    if len(hours[k]) == 1:
        hours[k]='0'+ hours[k]
# print(hours)
minutes05 = [] 
for m in range(0,60,5):
    minutes05.append(str(m))
for k in range (len(minutes05)):
    if len(minutes05[k]) == 1:
        minutes05[k]='0'+ minutes05[k]
minutes05
date = '2020-10-16 {}:{}:00'
time_window = []
for i in range (0, len(hours)-1):
        #print('Time zone:',time[i])
        for j in range (0,len(minutes05)):
            if hours [i] =='23':
                break
            elif j+1 == len(minutes05):
                j = -1
                # Declare high low filters
                low, high = date.format(hours[i],minutes05[j]), date.format(hours[i+1],minutes05[0])
                #print(low,high)
            else:
                # Declare high low filters
                low, high = date.format(hours[i],minutes05[j]), date.format(hours[i],minutes05[j+1])
                #print(low,high)
#             trips_temp = trips.loc[(trips['timestamp']>=low) & (trips['timestamp']<high)]
            time_window.append([low,high])
time = []
for i in time_window[0:-1]:
    time.append(i[0])
time.append(time_window[-1][0])
time.append(time_window[-1][1])     

In [8]:
# Range slider
# time_filter = dispatches.timestamp.unique().tolist()
time_filter = time
l = Layout(flex='0 1 0', height='60px', min_height='40px', width='90%')
selection_range_slider = widgets.SelectionRangeSlider(
    options=time_filter,
    index=(0, 1),
    description='Time Range',
    disabled=False,
    orientation='horizontal',
    layout=l
)

In [9]:
def dispatches_filter(date_range): #Φιλτράρει Dataframes dipatches_final και trips_gdf και  και τυπώνει χάρτη
    dis_visual = dispatches.loc[(dispatches.loc[:,'timestamp'] < date_range[1]) & (dispatches.loc[:,'timestamp'] >= date_range[0])].copy()
    total_visual = trips_gdf.loc[(trips_gdf.loc[:,'timestamp'] < date_range[1]) & (trips_gdf.loc[:,'timestamp'] > date_range[0])].copy()
    display(plot_map2(dis_visual, total_visual))

# Map demo
---

> - Map depicts total trip requests in a specific time window (e.x. 5 minutes) and total pairs created in it. <br>
> - `Antpaths` represent the shared trips. <br>
> - `Solid lines` represent single trips. <br>
> - `Solid circles` represent origin of a trip request and *transparent circles* destination. <br>

*In initial version of the notebook time windows can be selected manually with a time slider* <br>
*Database with trip requests can not be shared*

In [11]:
dis_visual = dispatches.loc[(dispatches.loc[:,'timestamp'] < '2020-10-16 08:05:00') & (dispatches.loc[:,'timestamp'] >= '2020-10-16 08:00:00')].copy()
total_visual = trips_gdf.loc[(trips_gdf.loc[:,'timestamp'] < '2020-10-16 08:05:00') & (trips_gdf.loc[:,'timestamp'] > '2020-10-16 08:00:00')].copy()
display(plot_map2(dis_visual, total_visual))

In [10]:
widgets.interact_manual(
    dispatches_filter,
    date_range=selection_range_slider
)

interactive(children=(SelectionRangeSlider(description='Time Range', index=(0, 1), layout=Layout(flex='0 1 0',…

<function __main__.dispatches_filter(date_range)>